In [1]:
from PIL import Image
import random
import os

### Setting variables

- base_image_base_dir : game screenshots dir
- asset_image_base_dir : game asset images dir
- result_base_dir : `base images` * `asset images` + `txt file which have YOLO format coordinates`
- resolution : game screenshot resolution

In [7]:
base_image_base_dir = r"data\raw_images\base"
asset_image_base_dir = r"data\raw_images\asset"
result_base_dir = r"data\learng_data"

base_images = os.listdir(base_image_base_dir)
asset_images = os.listdir(asset_image_base_dir)

resolution = (1920, 1080)

In [8]:
class ImageCoord():
    def __init__(self, x, y, w, h, cls = 0):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.x2 = self.x + self.w
        self.y2 = self.y + self.h
        self.cls = cls
        
    def __str__(self):
        return f"{self.x}, {self.y}, {self.w}, {self.h}"

    def get_VOLO_format(self):
        return f"{self.cls} {(self.x + (self.w / 2)) / resolution[0]:.6f} {(self.y + (self.h / 2))/resolution[1]:.6f} {self.w/resolution[0]:.6f} {self.h/resolution[1]:.6f}"

    def is_inside_rect(self, x, y):
        if self.x <= x <= self.x+self.w and self.y <= y <= self.y+self.h:
            return True
        else:
            return False

    def is_overlap(self, coord):
        if self.is_inside_rect(coord.x, coord.y) or \
        self.is_inside_rect(coord.x, coord.y2) or \
        self.is_inside_rect(coord.x2, coord.y) or \
        self.is_inside_rect(coord.x2, coord.y2) or \
        coord.is_inside_rect(self.x, self.y) or \
        coord.is_inside_rect(self.x, self.y2) or \
        coord.is_inside_rect(self.x2, self.y) or \
        coord.is_inside_rect(self.x2, self.y2):
            return True
        else:
            return False
           

### Make labels.txt

- Using game asset images's file name, generate `labels.txt`

- ex) If there are `A.png`, `B.png`, `C.png` in the `raw_images/asset`, a `labels.txt` file is created with "A", "B", "C" on each line.

In [9]:
def create_labels_txt():
    with open(os.path.join(result_base_dir, "labels.txt"), "w") as f:
        for image in asset_images:
            f.write(image.split(".")[0])
            f.write("\n")

create_labels_txt()

### Make `generated images`

#### - Using `game asset images` and `base images`, make `generated images`

In [10]:
for loop in range(5):
    for base_image in base_images:
        inserted_image_coord = []
        image_a_path = os.path.join(base_image_base_dir, base_image)
        image_a = Image.open(image_a_path)
        a_width, a_height = image_a.size

        
        for i, asset_image in enumerate(asset_images):
            image_b_path = os.path.join(asset_image_base_dir, asset_image)
            image_b = Image.open(image_b_path)
            b_width, b_height = image_b.size
    
            x_max = a_width - b_width
            y_max = a_height - b_height
            
            if x_max < 0 or y_max < 0:
                raise ValueError("B.png가 A.png보다 커서 삽입할 수 없습니다!")
    
            cnt = 0
            fail_cnt = 0
            fail_cnt_threshold = 10
            asset_cnt = random.randint(0, 10)
            while cnt < asset_cnt:
                # 랜덤 좌표 생성
                x1 = random.randint(0, x_max)
                y1 = random.randint(0, y_max)
                x2 = x1+b_width
                y2 = y1+b_height
                temp_coord = ImageCoord(x1, y1, b_width, b_height, i)
            
                check_result = False
                for coord in inserted_image_coord:
                    if coord.is_overlap(temp_coord):
                        check_result = True
                        fail_cnt += 1
                        break

                if fail_cnt >= fail_cnt_threshold:
                    del temp_coord
                    break
            
                if check_result:
                    del temp_coord
                    continue
                
                # B 이미지를 A 이미지 위에 붙이기
                cnt += 1
                image_a.paste(image_b, (x1, y1), image_b.convert("RGBA"))
                inserted_image_coord.append(temp_coord)

        out_image_file_name = f"{loop}_{base_image.split(".")[0]}"
        # 결과 이미지 저장
        output_path = os.path.join(result_base_dir, out_image_file_name + ".jpg")
        image_a.save(output_path)
    
        # 좌표, 클래스 저장    
        annotation_file_path = os.path.join(result_base_dir, out_image_file_name + ".txt")
        with open(annotation_file_path, 'w') as f:
            for coord in inserted_image_coord:
                # 좌표 기록
                f.write(coord.get_VOLO_format())
                f.write("\n")